# WEVT Templates for Offline EVTX Analysis

## The Problem

Windows event logs store records as **BinXML**, but the XML "shape" (element names, substitution slots) often lives in **provider templates** embedded in Windows binaries (`WEVT_TEMPLATE` PE resources), not in the EVTX file itself.

This breaks when:
- You're analyzing logs **offline** (different machine, Linux, macOS)
- You've **carved** records from disk/memory (no chunk context)
- The provider DLL was **deleted/replaced** (malware cleanup, updates)

## The Solution: WEVT Cache

Build a portable template cache from Windows binaries **once**, use it **anywhere**:

```
1. Collect binaries from target system (System32, SysWOW64)
2. Extract WEVT_TEMPLATE resources → .wevtcache file
3. Use cache as fallback when parsing EVTX offline
```

In [ ]:
from pathlib import Path
import evtx
from evtx import PyEvtxParser, WevtCache

print(f"evtx version: {getattr(evtx, '__version__', 'dev')}")

# Locate repo samples
def find_repo_root(start: Path) -> Path:
    """Find the main evtx repo root (not pyevtx-rs)."""
    cur = start.resolve()
    for _ in range(20):
        # Main repo has evtx-wasm/ and samples/dlls/
        if (cur / "evtx-wasm").exists() and (cur / "samples" / "dlls").exists():
            return cur
        cur = cur.parent
    raise RuntimeError("Could not locate evtx repo root")

REPO = find_repo_root(Path.cwd())
SAMPLES = REPO / "samples"
PY_SAMPLES = REPO / "external" / "pyevtx-rs" / "samples"
print(f"Repo: {REPO}")

---

## 1. Building a WEVT Cache

Extract templates from Windows binaries:

In [ ]:
# On Windows - build cache from System32:
#
# cache = WevtCache()
# cache.add_dir(r"C:\Windows\System32", recursive=True, extensions="exe,dll,sys")
# cache.dump("system.wevtcache", overwrite=True)
#
# Key DLLs for common events:
#   - adtschema.dll  → Security-Auditing (4624, 4625, 4688, etc.)
#   - services.exe   → SCM (7045, 7036, 7040)
#   - wevtsvc.dll    → Event Log service
#   - lsasrv.dll     → LSA events

print("Example: cache.add_dll(r'C:\\Windows\\System32\\adtschema.dll')")

---

## 2. Using a Pre-built Cache

Load a `.wevtcache` file built from a real Windows 11 system:

In [ ]:
# Load or build a cache
#
# Option 1: Load pre-built cache
# cache = WevtCache.load("windows.wevtcache")
#
# Option 2: Build from Windows System32
# cache = WevtCache()
# cache.add_dir(r"C:\Windows\System32", extensions="exe,dll,sys")
# cache.dump("windows.wevtcache", overwrite=True)

# For this demo, create empty cache (template resolution won't work)
cache = WevtCache()
print(f"Empty cache: {cache}")
print("\nTo get real templates, run on Windows:")
print('  cache.add_dll(r"C:\\Windows\\System32\\adtschema.dll")')

---

## 3. Template Resolution

Resolve template GUIDs from (provider, event_id, version):

In [ ]:
# Well-known provider GUIDs
PROVIDERS = {
    "Security-Auditing": "54849625-5478-4994-a5ba-3e3b0328c30d",
    "Service Control Manager": "555908d1-a6d7-4695-8e1e-26931d2012f4",
}

# Common security events
EVENTS = [
    ("Security-Auditing", 4624, 2, "Successful Logon"),
    ("Security-Auditing", 4625, 0, "Failed Logon"),
    ("Security-Auditing", 4688, 2, "Process Creation"),
    ("Security-Auditing", 4672, 0, "Special Privileges Assigned"),
    ("Security-Auditing", 4648, 0, "Explicit Credentials Logon"),
    ("Service Control Manager", 7045, 0, "Service Installed"),
    ("Service Control Manager", 7036, 0, "Service State Change"),
]

print("Template resolution:")
print("-" * 65)
for provider_name, event_id, version, desc in EVENTS:
    try:
        guid = cache.resolve_template_guid(
            PROVIDERS[provider_name], event_id, version
        )
        print(f"✓ {event_id:>5} v{version} {desc:<30} → {guid[:12]}...")
    except KeyError:
        print(f"✗ {event_id:>5} v{version} {desc:<30} → not in cache")

---

## 4. Template Rendering

Render a template with substitution values - useful for carved records:

In [ ]:
try:
    # Get Event 7045 (Service Installed) template
    template_guid = cache.resolve_template_guid(
        PROVIDERS["Service Control Manager"],
        7045,
        0
    )
    print(f"Template GUID: {template_guid}\n")

    # Render with suspicious service installation values
    xml = cache.render_template_xml(template_guid, [
        "WindowsUpdateService",           # ServiceName (masquerading)
        r"C:\Windows\Temp\svchost.exe",   # ImagePath (suspicious location)
        "share process",                   # ServiceType
        "auto start",                      # StartType
        "LocalSystem"                      # AccountName
    ])

    print("Rendered XML (suspicious service):")
    print(xml)
except KeyError:
    print("Template not in cache - build cache from Windows System32")

---

## 5. Parsing with Cache Fallback

Use the cache when parsing EVTX files:

In [ ]:
# Parse with cache as fallback (used when embedded templates are missing)
security_log = SAMPLES / "security.evtx"

if security_log.exists():
    parser = PyEvtxParser(
        str(security_log),
        wevt_cache=cache  # Cache used as fallback when templates missing
    )

    print("Sample records with cache:")
    for i, rec in enumerate(parser.records()):
        if i >= 2:
            break
        print(f"\nRecord {rec['event_record_id']} @ {rec['timestamp']}")
        print(rec['data'][:400] + "...")
else:
    print(f"Sample log not found: {security_log}")

---

## 6. DFIR Workflow: Disk Image Analysis

```python
from pathlib import Path
from evtx import PyEvtxParser, WevtCache

IMAGE = Path("/mnt/suspect_image")
SYSTEM32 = IMAGE / "Windows/System32"
LOGS = IMAGE / "Windows/System32/winevt/Logs"

# 1. Build cache from mounted image
cache = WevtCache()
cache.add_dir(SYSTEM32, recursive=True, extensions="exe,dll,sys")
cache.dump("suspect.wevtcache", overwrite=True)

# 2. Parse Security log with cache
parser = PyEvtxParser(
    str(LOGS / "Security.evtx"),
    wevt_cache="suspect.wevtcache"
)

# 3. Hunt for suspicious events
for rec in parser.records_json():
    data = rec["data"]
    # Lateral movement indicators
    if '"EventID":4624' in data and '"LogonType":10' in data:
        print(f"RDP logon: {rec['timestamp']}")
        print(data)
```

---

## 7. CLI: `evtx_dump extract-wevt-templates`

```bash
# Extract from specific files
evtx_dump extract-wevt-templates \
    --input /mnt/image/Windows/System32/adtschema.dll \
    --input /mnt/image/Windows/System32/services.exe \
    --output system.wevtcache

# Extract from entire directory
evtx_dump extract-wevt-templates \
    --glob "/mnt/image/Windows/System32/**/*.{exe,dll,sys}" \
    --output system.wevtcache \
    --overwrite

# Use cache when parsing
evtx_dump --wevt-cache system.wevtcache /path/to/Security.evtx
```

---

## 8. Manifest Introspection (`evtx.wevt`)

For low-level exploration of WEVT_TEMPLATE resources, use `evtx.wevt` to parse raw CRIM blobs:

In [ ]:
from evtx.wevt import Manifest

# Load a CRIM blob from a Windows binary's WEVT_TEMPLATE resource
# You can extract this using pefile or the evtx_dump CLI
services_exe = SAMPLES / "dlls" / "services.exe"

if services_exe.exists():
    # Extract WEVT_TEMPLATE resource (requires pefile)
    try:
        import pefile
        pe = pefile.PE(str(services_exe))
        crim_blob = None
        for entry in pe.DIRECTORY_ENTRY_RESOURCE.entries:
            if hasattr(entry, 'name') and entry.name and str(entry.name) == 'WEVT_TEMPLATE':
                for rid in entry.directory.entries:
                    for rlang in rid.directory.entries:
                        rva = rlang.data.struct.OffsetToData
                        size = rlang.data.struct.Size
                        crim_blob = pe.get_memory_mapped_image()[rva:rva+size]
                        break

        if crim_blob:
            manifest = Manifest.parse(crim_blob)
            print(f"Parsed {len(crim_blob):,} byte CRIM blob from services.exe\n")

            for provider in manifest.providers:
                print(f"Provider: {provider.identifier}")
                print(f"  Events: {len(provider.events)}")
                print(f"  Templates: {len(provider.templates)}")

                # Show first few events
                for event in provider.events[:3]:
                    print(f"    Event {event.identifier} v{event.version}", end="")
                    if event.template_offset:
                        tpl = provider.get_template_by_offset(event.template_offset)
                        if tpl:
                            print(f" → {len(tpl.items)} items")
                        else:
                            print()
                    else:
                        print(" (no template)")
                if len(provider.events) > 3:
                    print(f"    ... and {len(provider.events) - 3} more events")
    except ImportError:
        print("Install pefile to extract WEVT_TEMPLATE: pip install pefile")
else:
    print(f"services.exe not found at {services_exe}")

In [ ]:
# Render a template's BinXML structure (with {sub:N} placeholders)
if services_exe.exists() and 'manifest' in dir():
    for provider in manifest.providers:
        if provider.templates:
            template = provider.templates[0]
            print(f"Template: {template.identifier}")
            print(f"Items: {len(template.items)}")
            for item in template.items[:5]:
                print(f"  - {item.name or '(unnamed)'}: in={item.input_data_type} out={item.output_data_type}")

            print(f"\nBinXML (first 500 chars):")
            xml = template.to_xml()
            print(xml[:500])
            if len(xml) > 500:
                print("...")
            break

---

## API Reference

### `evtx` module (parsing & cache)

| API | Purpose |
|-----|------|
| `PyEvtxParser(path, wevt_cache=...)` | Parse EVTX with optional cache fallback |
| `WevtCache()` | Create empty in-memory cache |
| `WevtCache.load(path)` | Load saved `.wevtcache` file |
| `cache.add_dll(path)` | Extract templates from one PE |
| `cache.add_dir(path, ...)` | Bulk extract from directory |
| `cache.dump(path)` | Save to portable `.wevtcache` |
| `cache.resolve_template_guid(...)` | (provider, event_id, version) → template_guid |
| `cache.render_template_xml(...)` | Render template with substitution values |
| `cache.render_record_xml(...)` | End-to-end offline rendering |

### `evtx.wevt` module (manifest introspection)

| Class/Method | Description |
|--------------|-------------|
| `Manifest.parse(bytes)` | Parse a raw CRIM blob |
| `manifest.providers` | List of `Provider` objects |
| `provider.identifier` | Provider GUID string |
| `provider.events` | List of `Event` definitions |
| `provider.templates` | List of `Template` objects |
| `provider.get_template_by_offset(int)` | Lookup template by offset |
| `event.identifier` | Event ID |
| `event.version` | Event version |
| `event.template_offset` | Offset to template (or None) |
| `template.identifier` | Template GUID string |
| `template.items` | List of `TemplateItem` slots |
| `template.to_xml(ansi_codec=None)` | Render BinXML with placeholders |
| `item.name` | Substitution slot name (or None) |
| `item.input_data_type` | Input data type code |
| `item.output_data_type` | Output data type code |